In [1]:
# Final with GUI working

import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
import tkinter as tk
from PIL import Image, ImageTk

class Application:
    def __init__(self):
        self.vs = cv2.VideoCapture(0)
        self.root = tk.Tk()
        self.root.title("Sign language to Text Converter")
        self.root.geometry("1200x1100")

        self.panel = tk.Label(self.root)
        self.panel.place(x=135, y=60, width=470, height=470)

        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=690, y=60, width=310, height=310)

        self.T = tk.Label(self.root)
        self.T.place(x=31, y=17)
        self.T.config(text="Sign Language to Text", font=("Courier", 25, "bold"))

        self.panel3 = tk.Label(self.root)
        self.panel3.place(x=500, y=640)

        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=640)
        self.T1.config(text="Character:", font=("Courier", 40, "bold"))

        self.character_label = tk.Label(self.root)
        self.character_label.place(x=350, y=640)
        self.character_label.config(font=("Courier", 40, "bold"))

        self.panel4 = tk.Label(self.root)
        self.panel4.place(x=220, y=700)

        self.T2 = tk.Label(self.root)
        self.T2.place(x=10, y=700)
        self.T2.config(text="Word:", font=("Courier", 40, "bold"))

        self.word_label = tk.Label(self.root)
        self.word_label.place(x=350, y=700)
        self.word_label.config(font=("Courier", 40, "bold"))

        self.panel5 = tk.Label(self.root)
        self.panel5.place(x=350, y=760)

        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=760)
        self.T3.config(text="Sentence:", font=("Courier", 40, "bold"))

        self.current_character = ""
        self.current_word = ""

        self.video_loop()

    def video_loop(self):
        detector = HandDetector(maxHands=1)
        classifier = Classifier("C:/Users/tanma/OneDrive/Desktop/Sign Language youtube/converted_keras/keras_model.h5",
                                "C:/Users/tanma/OneDrive/Desktop/Sign Language youtube/converted_keras/labels.txt")
        offset = 20
        imgSize = 300

        labels = ["0", "1", "2"]

        while True:
            success, img = self.vs.read()
            imgOutput = img.copy()
            hands, img = detector.findHands(img)
            if hands:
                hand = hands[0]
                x, y, w, h = hand['bbox']

                imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
                imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

                if not imgCrop.any():
                    continue

                imgCropShape = imgCrop.shape

                aspectRatio = h / w

                if aspectRatio > 1:
                    k = imgSize / h
                    wCal = math.ceil(k * w)
                    imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                    imgResizeShape = imgResize.shape
                    wGap = math.ceil((imgSize - wCal) / 2)
                    imgWhite[:, wGap:wCal + wGap] = imgResize
                    prediction, index = classifier.getPrediction(imgWhite, draw=False)
                    print(index)

                else:
                    k = imgSize / w
                    hCal = math.ceil(k * h)
                    imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                    imgResizeShape = imgResize.shape
                    hGap = math.ceil((imgSize - hCal) / 2)
                    imgWhite[hGap:hCal + hGap, :] = imgResize
                    prediction, index = classifier.getPrediction(imgWhite, draw=False)

                confidence_threshold = 0.9  # Adjust based on your observations
                if np.max(prediction) < confidence_threshold:
                    recognized_text = "Not Recognized"
                else:
                    recognized_text = labels[index]

                    # Update current character and word
                    if recognized_text != self.current_character:
                        self.current_character = recognized_text
                        self.current_word += recognized_text

                print(recognized_text)

                cv2.rectangle(imgOutput, (x - offset, y - offset - 50),
                              (x - offset + 90, y - offset - 50 + 50), (255, 0, 255), cv2.FILLED)
                cv2.putText(imgOutput, recognized_text, (x, y - 26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
                cv2.rectangle(imgOutput, (x - offset, y - offset),
                              (x + w + offset, y + h + offset), (255, 0, 255), 4)

                # Convert the OpenCV image to RGB format for use with Tkinter
                imgRGB = cv2.cvtColor(imgOutput, cv2.COLOR_BGR2RGB)
                # Convert the NumPy array to an ImageTk PhotoImage
                imgTk = ImageTk.PhotoImage(Image.fromarray(imgRGB))

                # Update the image in the Tkinter label
                self.panel.img = imgTk
                self.panel.config(image=imgTk)

                # Convert the imgWhite frame to RGB format for use with Tkinter
                imgWhiteRGB = cv2.cvtColor(imgWhite, cv2.COLOR_BGR2RGB)
                # Convert the NumPy array to an ImageTk PhotoImage
                imgWhiteTk = ImageTk.PhotoImage(Image.fromarray(imgWhiteRGB))

                # Update the imgWhite image in the Tkinter label
                self.panel2.img = imgWhiteTk
                self.panel2.config(image=imgWhiteTk)

                # Update the recognized text in the Tkinter label
                self.character_label.config(text=self.current_character)

                # Update the word in the Tkinter label
                self.word_label.config(text=self.current_word)

            # Update the Tkinter window
            self.root.update()

    def run(self):
        self.root.mainloop()

if __name__ == '__main__':
    app = Application()
    app.run()





1/1 [==============================] - 1s 1s/step
0
Not Recognized
1/1 [==============================] - 0s 39ms/step
1
Not Recognized
1/1 [==============================] - 0s 35ms/step
0
Not Recognized
1/1 [==============================] - 0s 36ms/step
1
Not Recognized
1/1 [==============================] - 0s 37ms/step
1
1
1/1 [==============================] - 0s 37ms/step
1
Not Recognized
1/1 [==============================] - 0s 37ms/step
2
Not Recognized
1/1 [==============================] - 0s 36ms/step
2
Not Recognized
1/1 [==============================] - 0s 34ms/step
1
1
1/1 [==============================] - 0s 36ms/step
1
1
1/1 [==============================] - 0s 34ms/step
1
1
1/1 [==============================] - 0s 37ms/step
1
1
1/1 [==============================] - 0s 37ms/step
1
1
1/1 [==============================] - 0s 34ms/step
1
1
1/1 [==============================] - 0s 35ms/step
1
1
1/1 [==============================] - 0s 36ms/step
1
1
1/1 [=======

1/1 [==============================] - 0s 39ms/step
1
Not Recognized
1/1 [==============================] - 0s 40ms/step
1
Not Recognized
1/1 [==============================] - 0s 42ms/step
1
1
1/1 [==============================] - 0s 42ms/step
2
Not Recognized
1/1 [==============================] - 0s 33ms/step
2
Not Recognized
1/1 [==============================] - 0s 36ms/step
1
Not Recognized
1/1 [==============================] - 0s 34ms/step
1
1
1/1 [==============================] - 0s 32ms/step
1
Not Recognized
1/1 [==============================] - 0s 35ms/step
1
1
1/1 [==============================] - 0s 33ms/step
1
Not Recognized
1/1 [==============================] - 0s 31ms/step
1
Not Recognized
1/1 [==============================] - 0s 35ms/step
1
Not Recognized
1/1 [==============================] - 0s 35ms/step
1
Not Recognized
1/1 [==============================] - 0s 37ms/step
1
Not Recognized
1/1 [==============================] - 0s 35ms/step
1
Not Recognized
1/1 

1/1 [==============================] - 0s 37ms/step
2
2
1/1 [==============================] - 0s 37ms/step
2
2
1/1 [==============================] - 0s 38ms/step
2
2
1/1 [==============================] - 0s 37ms/step
2
2
1/1 [==============================] - 0s 36ms/step
2
2
1/1 [==============================] - 0s 37ms/step
2
2
1/1 [==============================] - 0s 37ms/step
2
Not Recognized
1/1 [==============================] - 0s 37ms/step
1
1
1/1 [==============================] - 0s 37ms/step
1
Not Recognized
1/1 [==============================] - 0s 36ms/step
2
Not Recognized
1/1 [==============================] - 0s 38ms/step
1
Not Recognized
1/1 [==============================] - 0s 32ms/step
2
2
1/1 [==============================] - 0s 38ms/step
2
2
1/1 [==============================] - 0s 36ms/step
2
2
1/1 [==============================] - 0s 30ms/step
2
2
1/1 [==============================] - 0s 33ms/step
2
2
1/1 [==============================] - 0s 34ms/step


KeyboardInterrupt: 